<a href="https://colab.research.google.com/github/f3rry12/FGA-BigDataTraining/blob/main/Flask_preparingWepAppEnvironment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Web App dengan Framework Flask untuk persiapan dalam membangun Lingkungan atau Ekosistem Pengolahan Big Data Using Python



Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWj8xZYranWtGUDB_7n1hCjdF7HoFAE9LFFc6AhOeZekkOFU4HjkuGs
Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/My Drive/FGA Big Data Using Python - Kelas B - utk Peserta/Link Tempat Upload Tugas Minggu ke-3 Big Data Using Python Kelas B/Ferry Abu Darrin Al Ghifari_01123956151-24")
!pwd

/content/drive/.shortcut-targets-by-id/1XiNT60qajEv0-xmg6AS66hDsF00Wmpau/FGA Big Data Using Python - Kelas B - utk Peserta/Link Tempat Upload Tugas Minggu ke-3 Big Data Using Python Kelas B/Ferry Abu Darrin Al Ghifari_01123956151-24


In [6]:
!ls

Data_Generate_EA.csv  templates
fga_big_data.db       Tugas_Minggu_3_Big_Data_Using_Python_Part1of2.ipynb.ipynb
static		      Tugas_Minggu_3_Big_Data_Using_Python_Part2of2.ipynb.ipynb


In [7]:
!pip install flask-ngrok
!pip install flask==0.12.2  # Newer versions of flask don't work in Colab
                            # See https://github.com/plotly/dash/issues/257

# !pip install flask
!pip install SQLAlchemy
!pip install Flask-SQLAlchemy

     |████████████████████████████████| 83 kB 1.3 MB/s 
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4


In [23]:
# flask_ngrok_example.py
from flask import Flask,render_template,flash, redirect,url_for,session,logging,request
from flask_sqlalchemy import SQLAlchemy
from flask_ngrok import run_with_ngrok

# !pip install SQLAlchemy
# !pip install Flask-SQLAlchemy

# app = Flask(__name__)
# app = Flask(__name__, static_url_path='/static')
app = Flask(__name__, static_folder='static')
app.secret_key = 'FGAferryNotSecret'
run_with_ngrok(app)  # Start ngrok when app is run

# app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////fga_big_data.db'
db = SQLAlchemy(app)

class user(db.Model):
  # __tablename__ = 'user'
  id = db.Column(db.Integer, primary_key=True)
  # Mail 	Password 	Name 	Level
  Mail = db.Column(db.Text) # sbg Username
  Password = db.Column(db.Text)
  Name = db.Column(db.Text)
  Level = db.Column(db.Text)

  # username = Mail
  # password = Password

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/logout",methods=["GET", "POST"])
def logout():
  session.pop('mail', None)
  session.pop('name', None)
  return redirect(url_for("index"))

@app.route("/login",methods=["GET", "POST"])
def login():
    msg = ""
    if request.method == "POST":
        mail = request.form["mail"]
        passw = request.form["passw"]
        
        login = user.query.filter_by(Mail=mail, Password=passw).first()
        print(login)
        if login is not None:
            session['name'] = login.Name
            session['mail'] = login.Mail

            return redirect(url_for("bigdataApps"))
        elif login is None:
            msg = "Masukkan Username (Email) dan Password dgn Benar"
            print(msg)

    return render_template("login.html", msg = msg)

@app.route("/register", methods=["GET", "POST"])
def register():
    if request.method == "POST":
        mail = request.form['mail']
        uname = request.form['uname']
        passw = request.form['passw']

        register = user(Mail = mail, Name = uname, Password = passw)
        # register = user(Mail = mail, Password = passw)
        db.session.add(register)
        db.session.commit()

        return redirect(url_for("login"))
    return render_template("register.html")

@app.route("/bigdataApps", methods=["GET", "POST"])
def bigdataApps():
  if request.method == 'POST':
    import pandas as pd
    import numpy as np
    dataset = request.FILES['inputDataset']
    # dataset = request.files['inputDataset']
    #dataset = 'dataset_dump.csv'

    persentase_data_training = 90
    banyak_fitur = int(request.POST['banyakFitur'])
    banyak_hidden_neuron = int(request.POST['banyakHiddenNeuron'])
    dataset = pd.read_csv(dataset, delimiter=';', names = ['Tanggal', 'Harga'], usecols=['Harga'])
    minimum = int(dataset.min()-10000)
    maksimum = int(dataset.max()+10000)
    new_banyak_fitur = banyak_fitur + 1
    hasil_fitur = []
    for i in range((len(dataset)-new_banyak_fitur)+1):
      kolom = []
      j = i
      while j < (i+new_banyak_fitur):
        kolom.append(dataset.values[j][0])
        j += 1
      hasil_fitur.append(kolom)
    hasil_fitur = np.array(hasil_fitur)
    data_normalisasi = (hasil_fitur - minimum)/(maksimum - minimum)
    data_training = data_normalisasi[:int(persentase_data_training*len(data_normalisasi)/100)]
    data_testing = data_normalisasi[int(persentase_data_training*len(data_normalisasi)/100):]
    
    #Training
    bobot = np.random.rand(banyak_hidden_neuron, banyak_fitur)
    bias = np.random.rand(banyak_hidden_neuron)
    h = 1/(1 + np.exp(-(np.dot(data_training[:, :banyak_fitur], np.transpose(bobot)) + bias)))
    h_plus = np.dot(np.linalg.inv(np.dot(np.transpose(h),h)),np.transpose(h))
    output_weight = np.dot(h_plus, data_training[:, banyak_fitur])
    
    #Testing
    h = 1/(1 + np.exp(-(np.dot(data_testing[:, :banyak_fitur], np.transpose(bobot)) + bias)))
    predict = np.dot(h, output_weight)
    predict = predict * (maksimum - minimum) + minimum
    
    #MAPE
    aktual = np.array(hasil_fitur[int(persentase_data_training*len(data_normalisasi)/100):, banyak_fitur])
    mape = np.sum(np.abs(((aktual - predict)/aktual)*100))/len(predict)
    return render_template('bigdataApps.html', {
        'y_aktual' : list(aktual),
        'y_prediksi' : list(predict),
        'mape' : mape
        })
  else:
    return render_template('bigdataApps.html')

if __name__ == '__main__':
  db.create_all()
  app.run()  # If address is in use, may need to terminate other sessions:
             # Runtime > Manage Sessions > Terminate Other Sessions

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/usr/local/lib/python3.7/dist-packages/flask_sqlalchemy/__init__.py:873: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c34937fa0d20.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Aug/2021 00:38:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 00:38:50] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Aug/2021 00:38:52] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 00:38:58] "GET /register HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 00:39:04] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 00:39:19] "POST /login HTTP/1.1" 302 -


<user 1>


127.0.0.1 - - [09/Aug/2021 00:39:19] "GET /bigdataApps HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 00:39:20] "GET /static/css/chartist-plugin-tooltip.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 00:39:20] "GET /static/css/jquery.toast.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 00:39:20] "GET /static/css/sidebar-nav.min.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 00:39:20] "GET /static/css/morris.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 00:39:20] "GET /static/css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 00:39:20] "GET /static/css/chartist.min.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 00:39:20] "GET /static/css/fullcalendar.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 00:39:20] "GET /static/css/animate.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 00:39:20] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 00:39:20] "GET /static/css/spinners.css HTTP/1.1" 200 -
127.0.0.1 - - [09/Aug/2021 00:39:20] "GET /static/css/blue-dark.cs